In [ ]:
# 全体の満足度スコア分析
print("=== 満足度スコア分析 ===")

all_results = [result1, result2, result3, consolidation_result, result_question, result_appeal, result_final]
final_scores = None

for i, result in enumerate(all_results, 1):
    scores = result.get('satisfaction_scores', {})
    if scores:
        print(f"\\nTurn {i} 満足度:")
        for dimension, score in scores.items():
            print(f"  {dimension}: {score}")
        final_scores = scores

if final_scores:
    avg_score = sum(final_scores.values()) / len(final_scores)
    print(f"\\n最終平均満足度: {avg_score:.2f}")
    
    print("\\n各次元詳細:")
    for dim, score in final_scores.items():
        status = "✅ 良好" if score >= 0.7 else "⚠️ 要改善" if score >= 0.5 else "❌ 不良"
        print(f"  {dim}: {score:.2f} {status}")

print("\\n=== フロー完了 ===")
print("実行されたフロー:")
print("1. ✅ 基準の説明 (AI)")
print("2. ✅ ユーザー重視点の要約確認 (AI)")  
print("3. ✅ ユーザー回答 (User)")
print("4. ✅ Bot評価者観点整理 (AI)")
print("5. ✅ 質問・異議申し立て (User)")
print("6. ✅ 異議申し立て結果伝達 (AI)")
print("7. ✅ 最終結果要約 (AI)")

print(f"\\n総ターン数: {current_state['turn']}")
print(f"最終ステージ: {current_state['stage']}")
print(f"異議申し立て: {'あり' if current_state.get('appeal_made') else 'なし'}")

# フロントエンド対応チャットフロー テスト

フロントエンドの実際のフローに合わせたテストnotebook


In [ ]:
import sys
import os
from pathlib import Path
import json
from typing import Dict, Any

# プロジェクトルートをパスに追加
project_root = Path().cwd().parent
sys.path.insert(0, str(project_root))

# 環境変数読み込み
from dotenv import load_dotenv
load_dotenv(project_root / '.env')

# サービスimport
from src.services.langgraph_service import LangGraphService

print("✅ 初期化完了")

In [ ]:
# フロントエンドと同じデータ構造を準備
sample_session_data = {
    'student_info': {
        'name': '田中太郎',
        'student_id': 'S2024001',
        'detailed_scores': {
            '学業成績': 3.2,
            '研究能力': 2.8,
            'コミュニケーション': 3.5,
            'リーダーシップ': 2.3,
            '将来性': 3.0
        }
    },
    'decision_data': {
        'user_weights': {
            '学業成績': 30,
            '研究能力': 25,
            'コミュニケーション': 20,
            'リーダーシップ': 10,
            '将来性': 15
        },
        'bot_evaluators': {
            'bot1': {'decision': '不合格'},
            'bot2': {'decision': '不合格'},
            'bot3': {'decision': '合格'}
        }
    },
    'rule_summary': {
        'criteria': '5項目の加重平均による総合評価',
        'threshold_description': '各項目2.5点以上が基準',
        'evaluation_method': '重み配分に基づく加重平均と多数決'
    },
    'threshold': 2.5,
    'last_user_text': ''
}

# ユーザーの初期決定データ（フロントエンドのPhase1結果）
initial_decision_data = {
    'decision': '合格',
    'weights': {
        '学業成績': 30,
        '研究能力': 25,
        'コミュニケーション': 20,
        'リーダーシップ': 10,
        '将来性': 15
    },
    'reasoning': '学業成績と研究能力が基準を上回っており、将来性も期待できるため。'
}

# セッション状態初期化
initial_state = {
    'turn': 0,
    'stage': 0.0,
    'route': 'Rules',
    'appeal_made': False,
    'rules_shown': False,
    'questions': []
}

print("✅ テストデータ準備完了")
print(f"ユーザー決定: {initial_decision_data['decision']}")
print(f"重要度設定: {initial_decision_data['weights']}")

In [ ]:
# LangGraphService初期化
lg_service = LangGraphService(enable_logging=False, session_id="frontend_test")
current_state = initial_state.copy()

print("✅ LangGraphService初期化完了")

In [ ]:
# フローに従って: ユーザーの重視点の回答 + 品質チェック
print("=== ユーザー回答: 重視点について (品質チェック付き) ===")

user_criteria_response = "学業成績と研究能力を特に重視しています。学業成績30%、研究能力25%に設定しました。この学生は学業成績3.2、研究能力2.8と基準を上回っており、基礎的な学力がしっかりしていると判断しています。"

# 品質チェック（裏側で動作、ユーザーには見えない）
from src.services.simple_llm import SimpleLLMResponder
quality_checker = SimpleLLMResponder()

print("🔍 [内部] 日本語品質チェック実行中...")
try:
    quality_result = quality_checker.check_japanese_quality(user_criteria_response)
    print(f"🔍 [内部] 品質チェック結果: {quality_result}")
    
    if not quality_result.get("is_valid", True):
        print(f"⚠️ [内部] 品質チェック警告: {quality_result.get('reason', '不明')}")
    else:
        print("✅ [内部] 品質チェック通過")
except Exception as e:
    print(f"⚠️ [内部] 品質チェックエラー: {e}")

result3 = lg_service.execute_turn(
    message=user_criteria_response,
    decision=initial_decision_data['decision'],
    state=current_state,
    session_data=sample_session_data
)

current_state = result3['state']
print(f"👤 User: {user_criteria_response}")
print(f"🤖 AI (回答確認): {result3['response']}")
print(f"Turn: {result3['turn']}, Stage: {current_state['stage']}")
print("-" * 50)

In [ ]:
# フローに従って: ユーザーの重視点の回答
print("=== ユーザー回答: 重視点について ===")

user_criteria_response = "学業成績と研究能力を特に重視しています。学業成績30%、研究能力25%に設定しました。この学生は学業成績3.2、研究能力2.8と基準を上回っており、基礎的な学力がしっかりしていると判断しています。"

result3 = lg_service.execute_turn(
    message=user_criteria_response,
    decision=initial_decision_data['decision'],
    state=current_state,
    session_data=sample_session_data
)

current_state = result3['state']
print(f"👤 User: {user_criteria_response}")
print(f"🤖 AI (回答確認): {result3['response']}")
print(f"Turn: {result3['turn']}, Stage: {current_state['stage']}")
print("-" * 50)

In [ ]:
# フローに従って: bot評価者の観点整理と質問確認
print("=== AI: Bot評価者との観点整理と質問確認 ===")

# 自動的に観点整理を実行
consolidation_result = lg_service.execute_turn(
    message="観点を整理してください",
    decision=initial_decision_data['decision'],
    state=current_state,
    session_data=sample_session_data
)

current_state = consolidation_result['state']
print(f"🤖 AI (観点整理): {consolidation_result['response']}")
print(f"Turn: {consolidation_result['turn']}, Stage: {current_state['stage']}")
print("-" * 50)

In [ ]:
# フローに従って: ユーザーからの質問や異議申し立て
print("=== ユーザー: 質問・異議申し立て ===")

user_question = "リーダーシップの評価が2.3と低いのが気になります。bot評価者はこの点をどう判断しているのでしょうか？また、この評価を見直すことは可能でしょうか？"

result_question = lg_service.execute_turn(
    message=user_question,
    decision=initial_decision_data['decision'],
    state=current_state,
    session_data=sample_session_data
)

current_state = result_question['state']
print(f"👤 User: {user_question}")
print(f"🤖 AI (質問回答): {result_question['response']}")
print(f"Turn: {result_question['turn']}, Stage: {current_state['stage']}")
print("-" * 50)

In [ ]:
# フローに従って: 異議申し立て結果の伝達
print("=== AI: 異議申し立て結果伝達 ===")

appeal_message = "評価の見直しを要請します。リーダーシップの評価が低すぎると思います。"

result_appeal = lg_service.execute_turn(
    message=appeal_message,
    decision=initial_decision_data['decision'],
    state=current_state,
    session_data=sample_session_data
)

current_state = result_appeal['state']
print(f"👤 User (異議申し立て): {appeal_message}")
print(f"🤖 AI (異議申し立て結果): {result_appeal['response']}")
print(f"Turn: {result_appeal['turn']}, Stage: {current_state['stage']}")
print(f"Appeal Made: {current_state.get('appeal_made', False)}")
print("-" * 50)

In [ ]:
# フローに従って: 最終的な結果の要約
print("=== AI: 最終結果要約 ===")

final_summary_message = "分かりました。ありがとうございました。"

result_final = lg_service.execute_turn(
    message=final_summary_message,
    decision=initial_decision_data['decision'],
    state=current_state,
    session_data=sample_session_data
)

current_state = result_final['state']
print(f"👤 User: {final_summary_message}")
print(f"🤖 AI (最終要約): {result_final['response']}")
print(f"Turn: {result_final['turn']}, Stage: {current_state['stage']}")
print("-" * 50)